In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
path = os.path.abspath('/content/gdrive/Shared drives/2040_project/DenseNet_CIFAR10_weights')
# os.mkdir(os.path.join(path, 'color_4thIter'))
path = os.path.join(path, 'color_4thIter')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation, SpatialDropout2D
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, SeparableConv2D
from keras.layers import Concatenate
from keras.optimizers import Adam, RMSprop, SGD
from keras import regularizers

Using TensorFlow backend.


In [0]:
# # this part will prevent tensorflow to allocate all the avaliable GPU Memory
# # backend
 import tensorflow as tf
# from keras import backend as k

# # Don't pre-allocate memory; allocate as-needed
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True

# # Create a session with the above options specified.
# k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 100
l = 12
num_filter = 36 #added 24 more filters
compression = 0.5 
dropout_rate = 0.2

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 6s 0us/step


In [0]:
# Dense Block
# removed the dropout
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        #if dropout_rate>0:
         # Conv2D_3_3 = Dropout2D(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False, kernel_regularizer = regularizers.l1() ,padding='same')(relu)
    #if dropout_rate>0:
      #Conv2D_BottleNeck = Dropout2D(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
# converted the last Dense Layer to a Fully Convolution N/w as use of Dense Layer was prohibited
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    temp = Conv2D(num_classes, kernel_size = (2,2))(AvgPooling)
    output = Activation('softmax')(temp)
    flat = Flatten()(output)
    
    return flat

In [0]:
num_filter = 36
dropout_rate = 0.2
l= 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [0]:
model = Model(inputs=[input], outputs=[output])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 36)   972         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 36)   144         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 36)   0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [0]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rotation_range = 15, horizontal_flip = True, width_shift_range = 0.1, height_shift_range = 0.1, zoom_range = 0.2, shear_range = 15)
datagen.fit(x_train)

KeyboardInterrupt: ignored

In [0]:
from keras.callbacks import ModelCheckpoint, CSVLogger
ckpt = ModelCheckpoint(os.path.join(path, 'model.hdf5'), monitor = 'val_acc')
csv = CSVLogger(os.path.join(path, 'log.csv'), append = True)


In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(0.01, momentum = 0.7),
              metrics=['accuracy'])

In [0]:
# Tensor Board
root_logdir = os.path.join(os.curdir, "my_logs")
def get_run_logdir():
    import time
    run_id = time.strftime("BW_run_1in_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)
run_logdir = get_run_logdir()
run_logdir
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [0]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size), steps_per_epoch = x_train.shape[0]/batch_size, epochs = 30, validation_data =(x_test, y_test), callbacks = [csv, ckpt])
model.save_weights(os.path.join(path, '30epochs.h5'))


Epoch 1/30
391/390 [==============================] - 148s 377ms/step - loss: 5.6643 - accuracy: 0.3967 - val_loss: 3.3625 - val_accuracy: 0.1180
Epoch 2/30
391/390 [==============================] - 125s 319ms/step - loss: 1.8821 - accuracy: 0.4690 - val_loss: 2.3624 - val_accuracy: 0.3805
Epoch 3/30
391/390 [==============================] - 124s 318ms/step - loss: 1.6310 - accuracy: 0.5263 - val_loss: 1.7732 - val_accuracy: 0.4870
Epoch 4/30
391/390 [==============================] - 126s 322ms/step - loss: 1.5138 - accuracy: 0.5661 - val_loss: 1.6410 - val_accuracy: 0.5204
Epoch 5/30
391/390 [==============================] - 125s 320ms/step - loss: 1.4274 - accuracy: 0.5953 - val_loss: 1.5586 - val_accuracy: 0.5609
Epoch 6/30
391/390 [==============================] - 126s 323ms/step - loss: 1.3549 - accuracy: 0.6221 - val_loss: 1.4783 - val_accuracy: 0.5785
Epoch 7/30
391/390 [==============================] - 126s 323ms/step - loss: 1.2956 - accuracy: 0.6436 - val_loss: 1.7978 -

In [0]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size), steps_per_epoch = x_train.shape[0]/batch_size, epochs = 30, validation_data =(x_test, y_test), callbacks = [csv, ckpt])
model.save_weights(os.path.join(path, '60epochs.h5'))

Epoch 1/30
391/390 [==============================] - 126s 322ms/step - loss: 0.8042 - accuracy: 0.8157 - val_loss: 0.9239 - val_accuracy: 0.7803
Epoch 2/30
391/390 [==============================] - 126s 323ms/step - loss: 0.8039 - accuracy: 0.8170 - val_loss: 1.0763 - val_accuracy: 0.7523
Epoch 3/30
391/390 [==============================] - 125s 320ms/step - loss: 0.7913 - accuracy: 0.8202 - val_loss: 1.1709 - val_accuracy: 0.7323
Epoch 4/30
391/390 [==============================] - 125s 320ms/step - loss: 0.7767 - accuracy: 0.8253 - val_loss: 1.0659 - val_accuracy: 0.7555
Epoch 5/30
391/390 [==============================] - 125s 320ms/step - loss: 0.7714 - accuracy: 0.8285 - val_loss: 1.0963 - val_accuracy: 0.7338
Epoch 6/30
391/390 [==============================] - 126s 321ms/step - loss: 0.7639 - accuracy: 0.8294 - val_loss: 0.8742 - val_accuracy: 0.7910
Epoch 7/30
391/390 [==============================] - 125s 320ms/step - loss: 0.7604 - accuracy: 0.8294 - val_loss: 0.9012 -

In [0]:
#restoring the last model
from keras.models import load_model
model = load_model(os.path.join(path, 'model.hdf5'))
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rotation_range = 15, horizontal_flip = True, width_shift_range = 0.1, height_shift_range = 0.1, zoom_range = 0.2, shear_range = 15)
datagen.fit(x_train)
model.fit_generator(datagen.flow(x_train, y_train, batch_size), steps_per_epoch = x_train.shape[0]/batch_size, epochs = 30, validation_data =(x_test, y_test), callbacks = [csv, ckpt])
model.save_weights(os.path.join(path, '72epochs.h5'))

Epoch 1/30
391/390 [==============================] - 138s 353ms/step - loss: 0.6324 - accuracy: 0.8692 - val_loss: 0.8829 - val_accuracy: 0.8101
Epoch 2/30
391/390 [==============================] - 126s 321ms/step - loss: 0.6357 - accuracy: 0.8686 - val_loss: 0.6855 - val_accuracy: 0.8476
Epoch 3/30
391/390 [==============================] - 125s 320ms/step - loss: 0.6233 - accuracy: 0.8719 - val_loss: 0.7874 - val_accuracy: 0.8289
Epoch 4/30
391/390 [==============================] - 125s 319ms/step - loss: 0.6241 - accuracy: 0.8714 - val_loss: 0.8455 - val_accuracy: 0.8136
Epoch 5/30
391/390 [==============================] - 126s 322ms/step - loss: 0.6183 - accuracy: 0.8730 - val_loss: 0.7207 - val_accuracy: 0.8465
Epoch 6/30
391/390 [==============================] - 126s 322ms/step - loss: 0.6115 - accuracy: 0.8758 - val_loss: 0.7559 - val_accuracy: 0.8389
Epoch 7/30
391/390 [==============================] - 125s 321ms/step - loss: 0.6121 - accuracy: 0.8749 - val_loss: 0.7121 -

In [0]:
model.load_weights(os.path.join(path, '72epochs.h5'))
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rotation_range = 15, horizontal_flip = True, width_shift_range = 0.1, height_shift_range = 0.1, zoom_range = 0.2, shear_range = 15)
datagen.fit(x_train)
model.fit_generator(datagen.flow(x_train, y_train, batch_size), steps_per_epoch = x_train.shape[0]/batch_size, epochs = 30, validation_data =(x_test, y_test), callbacks = [csv, ckpt])
model.save_weights(os.path.join(path, '120epochs.h5'))


Epoch 1/30
391/390 [==============================] - 126s 321ms/step - loss: 0.5504 - accuracy: 0.8918 - val_loss: 0.8066 - val_accuracy: 0.8326
Epoch 2/30
391/390 [==============================] - 125s 320ms/step - loss: 0.5456 - accuracy: 0.8931 - val_loss: 0.7587 - val_accuracy: 0.8372
Epoch 3/30
391/390 [==============================] - 125s 320ms/step - loss: 0.5424 - accuracy: 0.8937 - val_loss: 0.6319 - val_accuracy: 0.8734
Epoch 4/30
391/390 [==============================] - 125s 320ms/step - loss: 0.5363 - accuracy: 0.8970 - val_loss: 0.8955 - val_accuracy: 0.8076
Epoch 5/30
391/390 [==============================] - 125s 321ms/step - loss: 0.5325 - accuracy: 0.8972 - val_loss: 0.6572 - val_accuracy: 0.8593
Epoch 6/30
391/390 [==============================] - 125s 321ms/step - loss: 0.5369 - accuracy: 0.8971 - val_loss: 0.7126 - val_accuracy: 0.8540
Epoch 7/30
391/390 [==============================] - 126s 322ms/step - loss: 0.5281 - accuracy: 0.8993 - val_loss: 0.7614 -

In [0]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size), steps_per_epoch = 1.5*x_train.shape[0]/batch_size, epochs = 30, validation_data =(x_test, y_test), callbacks = [csv, ckpt])
model.save_weights(os.path.join(path, '150epochs.h5'))

Epoch 1/30
586/585 [==============================] - 185s 316ms/step - loss: 0.4873 - accuracy: 0.9112 - val_loss: 0.6363 - val_accuracy: 0.8756
Epoch 2/30
586/585 [==============================] - 185s 316ms/step - loss: 0.4867 - accuracy: 0.9113 - val_loss: 0.9201 - val_accuracy: 0.8110
Epoch 3/30
586/585 [==============================] - 187s 318ms/step - loss: 0.4816 - accuracy: 0.9116 - val_loss: 0.6923 - val_accuracy: 0.8554
Epoch 4/30
586/585 [==============================] - 186s 317ms/step - loss: 0.4769 - accuracy: 0.9135 - val_loss: 0.6976 - val_accuracy: 0.8561
Epoch 5/30
586/585 [==============================] - 185s 316ms/step - loss: 0.4742 - accuracy: 0.9136 - val_loss: 0.6754 - val_accuracy: 0.8638
Epoch 6/30
586/585 [==============================] - 185s 316ms/step - loss: 0.4792 - accuracy: 0.9127 - val_loss: 0.6648 - val_accuracy: 0.8646
Epoch 7/30
586/585 [==============================] - 185s 316ms/step - loss: 0.4679 - accuracy: 0.9166 - val_loss: 0.5972 -

In [0]:
model.load_weights(os.path.join(path, '150epochs.h5'))
keras.backend.set_value(model.optimizer.lr, .001)
model.fit_generator(datagen.flow(x_train, y_train, batch_size), steps_per_epoch = 3*x_train.shape[0]/batch_size, epochs = 20, validation_data =(x_test, y_test), callbacks = [csv, ckpt])
model.save_weights(os.path.join(path, '170epochs.h5'))#157 because it ran for 5 epochs before, as i forgot to update the learning rate

Epoch 1/20
1172/1171 [==============================] - 349s 298ms/step - loss: 0.2359 - accuracy: 0.9501 - val_loss: 0.3739 - val_accuracy: 0.9044
Epoch 2/20
1172/1171 [==============================] - 332s 284ms/step - loss: 0.2124 - accuracy: 0.9539 - val_loss: 0.3854 - val_accuracy: 0.9058
Epoch 3/20
1172/1171 [==============================] - 331s 283ms/step - loss: 0.2186 - accuracy: 0.9514 - val_loss: 0.4027 - val_accuracy: 0.9005
Epoch 4/20
1172/1171 [==============================] - 331s 282ms/step - loss: 0.2220 - accuracy: 0.9509 - val_loss: 0.4133 - val_accuracy: 0.8986
Epoch 5/20
1172/1171 [==============================] - 331s 282ms/step - loss: 0.2201 - accuracy: 0.9517 - val_loss: 0.4177 - val_accuracy: 0.8992
Epoch 6/20
1172/1171 [==============================] - 330s 282ms/step - loss: 0.2195 - accuracy: 0.9526 - val_loss: 0.4026 - val_accuracy: 0.9018
Epoch 7/20
1172/1171 [==============================] - 331s 282ms/step - loss: 0.2209 - accuracy: 0.9517 - val_

In [0]:
model.load_weights(os.path.join(path, '170epochs.h5'))
keras.backend.set_value(model.optimizer.momentum, 0.7)
keras.backend.set_value(model.optimizer.lr, 0.0005)
best_ckpt = ModelCheckpoint(os.path.join(path, 'best_model.h5'), monitor = 'val_acc', save_best_only = True)
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rotation_range = 15, horizontal_flip = True, width_shift_range = 0.1, height_shift_range = 0.1, zoom_range = 0.2, shear_range = 15)
datagen.fit(x_train)
model.fit_generator(datagen.flow(x_train, y_train, batch_size), steps_per_epoch = 3*x_train.shape[0]/batch_size, epochs = 20, validation_data =(x_test, y_test), callbacks = [csv, ckpt, best_ckpt])
model.save_weights(os.path.join(path, '190epochs.h5'))


Epoch 1/20
1172/1171 [==============================] - 332s 283ms/step - loss: 0.1761 - accuracy: 0.9603 - val_loss: 0.4114 - val_accuracy: 0.8985


/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


Epoch 2/20
1172/1171 [==============================] - 333s 284ms/step - loss: 0.1780 - accuracy: 0.9600 - val_loss: 0.4218 - val_accuracy: 0.8949
Epoch 3/20
1172/1171 [==============================] - 334s 285ms/step - loss: 0.1818 - accuracy: 0.9585 - val_loss: 0.4550 - val_accuracy: 0.8894
Epoch 4/20
1172/1171 [==============================] - 333s 284ms/step - loss: 0.1812 - accuracy: 0.9587 - val_loss: 0.3873 - val_accuracy: 0.9058
Epoch 5/20
1172/1171 [==============================] - 333s 284ms/step - loss: 0.1817 - accuracy: 0.9587 - val_loss: 0.3673 - val_accuracy: 0.9080
Epoch 6/20
1172/1171 [==============================] - 333s 284ms/step - loss: 0.1815 - accuracy: 0.9590 - val_loss: 0.3862 - val_accuracy: 0.9058
Epoch 7/20
1172/1171 [==============================] - 333s 284ms/step - loss: 0.1796 - accuracy: 0.9597 - val_loss: 0.4047 - val_accuracy: 0.9001
Epoch 8/20
1172/1171 [==============================] - 332s 284ms/step - loss: 0.1815 - accuracy: 0.9591 - val_

In [0]:
model.load_weights(os.path.join(path, '177epochs.h5'))
from keras.preprocessing.image import ImageDataGenerator
keras.backend.set_value(model.optimizer.momentum, 0.7)
keras.backend.set_value(model.optimizer.lr, 0.001)
best_ckpt = ModelCheckpoint(os.path.join(path, 'best_model.h5'), monitor = 'val_acc', save_best_only = True)
datagen = ImageDataGenerator(rotation_range = 15, horizontal_flip = True, width_shift_range = 0.1, height_shift_range = 0.1, zoom_range = 0.2, shear_range = 15)
datagen.fit(x_train)
model.fit_generator(datagen.flow(x_train, y_train, batch_size), steps_per_epoch = 3.5*x_train.shape[0]/batch_size, epochs = 10, validation_data =(x_test, y_test), callbacks = [csv, ckpt, best_ckpt])
model.save_weights(os.path.join(path, '187epochs.h5'))

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/10
1368/1367 [==============================] - 942s 689ms/step - loss: 0.1854 - acc: 0.9706 - val_loss: 0.4150 - val_acc: 0.9147
Epoch 2/10
1368/1367 [==============================] - 926s 677ms/step - loss: 0.1840 - acc: 0.9710 - val_loss: 0.4621 - val_acc: 0.9051
Epoch 3/10
1368/1367 [==============================] - 921s 673ms/step - loss: 0.1822 - acc: 0.9717 - val_loss: 0.4534 - val_acc: 0.9077
Epoch 4/10
1368/1367 [==============================] - 923s 675ms/step - loss: 0.1812 - acc: 0.9724 - val_loss: 0.4615 - val_acc: 0.9039
Epoch 5/10
1368/1367 [==============================] - 922s 674ms/step - loss: 0.1863 - acc: 0.9709 - val_loss: 0.4345 - val_acc: 0.9122
Epoch 6/10
1368/1367 [==============================] - 924s 676ms/step - loss: 0.1828 - acc: 0.9715 - val_loss: 0.4552 - val_acc: 0.9066
Epoch 7/10
1368/1367 [========================

In [0]:
model.load_weights(os.path.join(path, '187epochs.h5'))
datagen = ImageDataGenerator(rotation_range = 10, horizontal_flip = True, width_shift_range = 0.08, height_shift_range = 0.08, zoom_range = 0.15, shear_range = 10)
datagen.fit(x_train)
model.fit_generator(datagen.flow(x_train, y_train, batch_size), steps_per_epoch = 4*x_train.shape[0]/batch_size, epochs = 10, validation_data =(x_test, y_test), callbacks = [csv, ckpt, best_ckpt])
model.save_weights(os.path.join(path, '197epochs.h5'))

Epoch 1/10
1563/1562 [==============================] - 1057s 676ms/step - loss: 0.1200 - acc: 0.9864 - val_loss: 0.4136 - val_acc: 0.9134
Epoch 2/10
1563/1562 [==============================] - 1055s 675ms/step - loss: 0.1280 - acc: 0.9853 - val_loss: 0.4497 - val_acc: 0.9056
Epoch 3/10
1563/1562 [==============================] - 1053s 674ms/step - loss: 0.1266 - acc: 0.9857 - val_loss: 0.4673 - val_acc: 0.9032
Epoch 4/10
1563/1562 [==============================] - 1057s 676ms/step - loss: 0.1277 - acc: 0.9854 - val_loss: 0.4837 - val_acc: 0.8996
Epoch 5/10
1563/1562 [==============================] - 1064s 681ms/step - loss: 0.1270 - acc: 0.9862 - val_loss: 0.4422 - val_acc: 0.9090
Epoch 6/10
1563/1562 [==============================] - 1063s 680ms/step - loss: 0.1260 - acc: 0.9864 - val_loss: 0.4395 - val_acc: 0.9095
Epoch 7/10
1563/1562 [==============================] - 1064s 680ms/step - loss: 0.1272 - acc: 0.9860 - val_loss: 0.5276 - val_acc: 0.8968
Epoch 8/10
1563/1562 [=====

In [0]:
model.load_weights(os.path.join(path, '197epochs.h5'))
keras.backend.set_value(model.optimizer.lr, 0.0001)
bacth_size = 256
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rotation_range = 5, horizontal_flip = True, width_shift_range = 0.05, height_shift_range = 0.05, shear_range = 5)
datagen.fit(x_train)
best_ckpt = ModelCheckpoint(os.path.join(path, 'best_model.h5'), monitor = 'val_acc', save_best_only = True)
model.fit_generator(datagen.flow(x_train, y_train, batch_size), steps_per_epoch = 3*x_train.shape[0]/batch_size, epochs = 10, validation_data =(x_test, y_test), callbacks = [csv, ckpt, best_ckpt])
model.save_weights(os.path.join(path, '207epochs.h5'))

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/10
1172/1171 [==============================] - 822s 702ms/step - loss: 0.0498 - acc: 0.9969 - val_loss: 0.3443 - val_acc: 0.9235
Epoch 2/10
1172/1171 [==============================] - 806s 687ms/step - loss: 0.0418 - acc: 0.9972 - val_loss: 0.3436 - val_acc: 0.9238
Epoch 3/10
1172/1171 [==============================] - 805s 687ms/step - loss: 0.0398 - acc: 0.9973 - val_loss: 0.3457 - val_acc: 0.9222
Epoch 4/10
1172/1171 [==============================] - 805s 687ms/step - loss: 0.0384 - acc: 0.9972 - val_loss: 0.3404 - val_acc: 0.9242
Epoch 5/10
1172/1171 [==============================] - 806s 687ms/step - loss: 0.0373 - acc: 0.9972 - val_loss: 0.3404 - val_acc: 0.9234
Epoch 6/10
1172/1171 [==============================] - 806s 687ms/step - loss: 0.0372 - acc: 0.9969 - val_loss: 0.3386 - val_acc: 0.9223
Epoch 7/10
 740/1171 [=================>......

KeyboardInterrupt: ignored

In [0]:
keras.backend.set_value(model.optimizer.lr, 0.0001)
keras.backend.set_value(model.optimizer.momentum, 0.5)
bacth_size = 512
datagen = ImageDataGenerator(rotation_range = 5, horizontal_flip = True, width_shift_range = 0.05, height_shift_range = 0.05, shear_range = 5)
datagen.fit(x_train)
model.fit_generator(datagen.flow(x_train, y_train, batch_size), steps_per_epoch = 3*x_train.shape[0]/batch_size, epochs = 5 validation_data =(x_test, y_test), callbacks = [csv, ckpt, best_ckpt])

Epoch 1/5
1172/1171 [==============================] - 807s 688ms/step - loss: 0.0342 - acc: 0.9971 - val_loss: 0.3361 - val_acc: 0.9227
Epoch 2/5
1172/1171 [==============================] - 804s 686ms/step - loss: 0.0335 - acc: 0.9973 - val_loss: 0.3415 - val_acc: 0.9216
Epoch 3/5
1172/1171 [==============================] - 804s 686ms/step - loss: 0.0335 - acc: 0.9970 - val_loss: 0.3467 - val_acc: 0.9207
Epoch 4/5
1172/1171 [==============================] - 805s 687ms/step - loss: 0.0338 - acc: 0.9970 - val_loss: 0.3481 - val_acc: 0.9211
Epoch 5/5
1172/1171 [==============================] - 805s 687ms/step - loss: 0.0334 - acc: 0.9969 - val_loss: 0.3397 - val_acc: 0.9218


In [0]:
model.save_weights(os.path.join(path, '208epochs.h5'))

In [0]:
model.evaluate(x_test, y_test)

10000/10000 [==============================] - 20s 2ms/step


[0.33972805423736574, 0.9218]